In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(1, os.getenv('LIBRARY_PATH'))
from datetime import date
from dateutil.relativedelta import relativedelta
import scrapper

In [2]:
url = "https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410022301&pickMembers%5B0%5D=2.2&pickMembers%5B1%5D=3.35&cubeTimeFrame.startMonth=06&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=20220601%2C20221001"

table = "1410022301"

# test = scrapper.simple_scrapper(url, [])


In [3]:

industries_dict = {'3.1': 'industrial_aggregate_including_unclassified_businesses', '3.2': 'industrial_aggregate_excluding_unclassified_businesses', '3.3': 'goods_producing_industries', '3.4': 'forestry_logging_and_support', '3.10': 'mining_quarrying_and_oil_and_gas_extraction', '3.17': 'utilities', '3.21': 'construction', '3.34': 'manufacturing', '3.143': 'service_producing_industries', '3.144': 'trade', '3.215': 'transportation_and_warehousing', '3.253': 'information_and_cultural_industries', '3.267': 'finance_and_insurance', '3.284': 'real_estate_and_rental_and_leasing', '3.295': 'professional_scientific_and_technical_services', '3.306': 'management_of_companies_and_enterprises', '3.321': 'educational_services', '3.331': 'health_care_and_social_assistance', '3.354': 'arts_entertainment_and_recreation', '3.367': 'accommodation_and_food_services', '3.377': 'other_services_except_public_administration', '3.393': 'public_administration'}
pick_members_2 = {"names": ['2.2'], "values": ['average_weekly_earnings']}
pick_members_3 = {"names": ['3.1', '3.2', '3.3', '3.4', '3.10', '3.17', '3.21', '3.34', '3.143', '3.144', '3.215', '3.253', '3.267', '3.284', '3.295', '3.306', '3.321', '3.331', '3.354', '3.367', '3.377', '3.393'],"values": ['industrial_aggregate_including_unclassified_businesses', 'industrial_aggregate_excluding_unclassified_businesses', 'goods_producing_industries', 'forestry_logging_and_support', 'mining_quarrying_and_oil_and_gas_extraction', 'utilities', 'construction', 'manufacturing', 'service_producing_industries', 'trade', 'transportation_and_warehousing', 'information_and_cultural_industries', 'finance_and_insurance', 'real_estate_and_rental_and_leasing', 'professional_scientific_and_technical_services', 'management_of_companies_and_enterprises', 'educational_services', 'health_care_and_social_assistance', 'arts_entertainment_and_recreation', 'accommodation_and_food_services', 'other_services_except_public_administration', 'public_administration'] }
pick_members_dict = {"&pickMembers%5B0%5D=": pick_members_2, "&pickMembers%5B1%5D=": pick_members_3}
filter_names = []

In [4]:
pid="1410022301"
today = date.today()
four_months_ago = date.today() - relativedelta(months=+4)
start_month=str(four_months_ago.month).zfill(2)
start_year=str(four_months_ago.year)
end_month=str(today.month).zfill(2)
end_year=str(today.year)
conn_string = 'mysql+pymysql://' + os.environ["USERNAME"] + ':' + os.environ["PASSWORD"] + '@' + os.environ["HOST"] + '/' + os.environ["DATABASE"] 

referencePeriods = start_year + start_month + "01" + "%2C" + end_year + end_month + "28" 
keys = list(pick_members_dict.keys())
values = list((pick_members_dict[key]["names"] for key in keys))

pick_members_1_dict = dict(zip(pick_members_dict["&pickMembers%5B0%5D="]["names"], pick_members_dict["&pickMembers%5B0%5D="]["values"]))
pick_members_2_dict = dict(zip(pick_members_dict["&pickMembers%5B1%5D="]["names"], pick_members_dict["&pickMembers%5B1%5D="]["values"]))
result = [(x, y) for x in values[0] for y in values[1]]


In [5]:
print(result)

[('2.2', '3.1'), ('2.2', '3.2'), ('2.2', '3.3'), ('2.2', '3.4'), ('2.2', '3.10'), ('2.2', '3.17'), ('2.2', '3.21'), ('2.2', '3.34'), ('2.2', '3.143'), ('2.2', '3.144'), ('2.2', '3.215'), ('2.2', '3.253'), ('2.2', '3.267'), ('2.2', '3.284'), ('2.2', '3.295'), ('2.2', '3.306'), ('2.2', '3.321'), ('2.2', '3.331'), ('2.2', '3.354'), ('2.2', '3.367'), ('2.2', '3.377'), ('2.2', '3.393')]


In [25]:
import pandas as pd
final_df = pd.DataFrame(columns=list(industries_dict.values()))

In [26]:


for x, y in result:
    estimate = pick_members_1_dict[x]
    industry = pick_members_2_dict[y]

    url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ start_month + '&cubeTimeFrame.startYear=' + start_year + '&cubeTimeFrame.endMonth=' + end_month +'&cubeTimeFrame.endYear=' + end_year + '&referencePeriods=' + referencePeriods
    test_df = scrapper.simple_scrapper(url, filter_names)
    df = test_df.iloc[-1]
    final_df[industry] = df
    latest_month = df["month"]
    print("Running for: " + estimate + " " + industry + " " + latest_month)


Running for: average_weekly_earnings industrial_aggregate_including_unclassified_businesses October 2022
Running for: average_weekly_earnings industrial_aggregate_excluding_unclassified_businesses October 2022
Running for: average_weekly_earnings goods_producing_industries October 2022
Running for: average_weekly_earnings forestry_logging_and_support October 2022
Running for: average_weekly_earnings mining_quarrying_and_oil_and_gas_extraction October 2022
Running for: average_weekly_earnings utilities October 2022
Running for: average_weekly_earnings construction October 2022
Running for: average_weekly_earnings manufacturing October 2022
Running for: average_weekly_earnings service_producing_industries October 2022
Running for: average_weekly_earnings trade October 2022
Running for: average_weekly_earnings transportation_and_warehousing October 2022
Running for: average_weekly_earnings information_and_cultural_industries October 2022
Running for: average_weekly_earnings finance_and_in

In [27]:
print(industry)

public_administration


In [28]:
final_df

,industrial_aggregate_including_unclassified_businesses,industrial_aggregate_excluding_unclassified_businesses,goods_producing_industries,forestry_logging_and_support,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,manufacturing,service_producing_industries,trade,...,finance_and_insurance,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,management_of_companies_and_enterprises,educational_services,health_care_and_social_assistance,arts_entertainment_and_recreation,accommodation_and_food_services,other_services_except_public_administration,public_administration
canada,0.0,1171.6,1436.42,1329.06,2302.65,2033.29,1454.4,1251.35,1114.36,905.05,...,1527.25,1264.34,1626.53,1531.65,1165.19,1030.65,729.05,460.18,994.41,1514.69
newfoundland_and_labrador,0.0,1165.18,1558.71,1153.48,2093.18,2096.02,1492.37,1300.89,1087.38,798.26,...,1420.83,1226.57,1636.99,1542.77,1369.79,1045.85,608.8,483.92,946.64,1409.92
prince_edward_island,0.0,984.5,1112.87,0.0,0.0,2014.21,1108.18,1046.06,960.3,779.92,...,1299.11,885.9,1420.38,1322.6,1137.19,971.46,618.04,445.48,864.22,1299.27
nova_scotia,0.0,1033.15,1244.89,0.0,1705.21,1903.89,1309.83,1103.39,999.16,810.9,...,1422.4,1045.14,1486.52,1464.64,1105.68,971.51,659.5,421.71,889.15,1329.2
new_brunswick,0.0,1068.44,1216.64,0.0,1595.1,2144.26,1241.82,1137.3,1035.66,785.81,...,1326.91,1089.85,1464.08,1506.86,1231.95,1032.49,799.05,457.92,839.98,1391.08
quebec,0.0,1123.07,1357.46,1230.59,2011.15,1958.87,1420.75,1241.21,1063.32,835.33,...,1420.14,1152.79,1544.87,1518.47,1118.51,1045.72,698.58,435.21,926.12,1403.12
ontario,0.0,1200.69,1393.91,1274.29,2035.26,2021.81,1463.64,1274.43,1157.95,957.71,...,1591.21,1292.59,1693.3,1534.77,1147.96,1037.66,773.4,460.66,1037.74,1609.45
manitoba,0.0,1075.2,1257.99,0.0,1884.56,0.0,1279.88,1163.61,1033.6,840.08,...,1519.33,1002.03,1425.52,1362.65,1192.31,914.78,737.59,409.57,899.71,1383.72
saskatchewan,0.0,1144.25,1595.76,0.0,2254.3,2095.28,1396.34,1280.6,1053.89,904.85,...,1443.92,1278.0,1469.75,1465.17,1141.16,980.49,613.1,397.22,942.07,1444.18
alberta,0.0,1259.32,1746.12,1377.59,2530.86,2108.79,1582.83,1346.06,1132.08,964.64,...,1582.89,1477.56,1662.98,1817.83,1207.5,1034.18,639.81,497.26,1027.02,1583.62


In [29]:
print(test_df)

    canada newfoundland_and_labrador prince_edward_island nova_scotia  \
0  1517.68                   1390.13              1283.48     1306.79   
1  1514.69                   1409.92              1299.27      1329.2   

  new_brunswick   quebec  ontario manitoba saskatchewan  alberta  \
0       1402.98  1387.01  1618.68  1376.21      1465.22  1583.13   
1       1391.08  1403.12  1609.45  1383.72      1444.18  1583.62   

  british_columbia    yukon northwest_territories  nunavut        date  \
0          1530.19  1596.43               1832.37  1725.48  2013-09-27   
1          1539.11  1627.74               1827.03  1657.92  2013-09-27   

            month  
0  September 2022  
1    October 2022  
